# Import Stuff

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import random as rd
import csv
import numpy as np
import math
import student_utils
import utils
import solver
import os
import output_validator
from queue import PriorityQueue
from itertools import permutations

# Building outputs from all algorithms

## Select the best output for each input

In [ ]:
def select_outputs(pickle_dir):
    pfiles = get_files_with_extension(pickle_dir, 'p')
    dic = {}
    meta_dic = {}
    for pf in pfiles:
        pobj = load_obj(pf)
        for item in pobj:
            # key=<./path/to/input_file>.in, value=(<./path/to/output_file.out>, cost)
            input_file = item[0]
            output_file = item[1]
            cost = item[2][0]
            if input_file in dic:
                dic[input_file].append((output_file, cost))
            else:
                dic[input_file] = [(output_file, cost)]
    if not os.path.isdir('final/outputs'):
        os.mkdir('final')
        os.mkdir('final/outputs')
    for lst in dic.values():
        best_output, best_cost = min(lst, key=lambda x: x[1])
        os.system('cp ' + best_output + ' final/outputs/')
        meta_dic[best_output] = best_cost
    save_obj(meta_dic, 'final/meta.p')
    write_data_to_file('final/meta.txt', meta_dic.items(), '\n')

In [ ]:
select_outputs('pickle_output')

# Rao drives every TA home: this is a TSP

## Nearest Neighbor Algorithm

In [ ]:
def nn_solve(list_of_locations, list_of_homes, starting_car_location, adjacency_matrix, params=[]):
    start_idx = list_of_locations.index(starting_car_location)
    homes_idx = [list_of_locations.index(h) for h in list_of_homes]
    graph1, msg1 = adjacency_matrix_to_graph(adjacency_matrix)
    path = []
    drop_offs = {}
    current_idx = start_idx
    while homes_idx:
        lst_dist = [(h_idx, nx.shortest_path_length(graph1, current_idx, h_idx, 'weight')) for h_idx in homes_idx]
        idx, _ = min(lst_dist, key=lambda x: x[1])
        shortest_path = nx.shortest_path(graph1, current_idx, idx, 'weight')
        shortest_path.pop()
        path.extend(shortest_path)
        drop_offs[idx] = [idx]
        current_idx = idx
        homes_idx.remove(idx)
    last_shortest_path = nx.shortest_path(graph1, current_idx, start_idx, 'weight')
    path.extend(last_shortest_path)
    return path, drop_offs

In [ ]:
# Test
data = read_file('test_inputs/27_50.in')
num_loc, num_house, lst_loc, lst_house, start_loc, adj_mat = data_parser(data)
nn_solve(lst_loc, lst_house, start_loc, adj_mat)

### Nearest Neighbor Algorithm with Different First Node After Starting Node

In [ ]:
def cost_of_solution_no_print(G, car_cycle, dropoff_mapping):
    cost = 0
    dropoffs = dropoff_mapping.keys()
    if not is_valid_walk(G, car_cycle):
        print('This is not a valid walk for the given graph.\n')
        sys.exit()

    if not car_cycle[0] == car_cycle[-1]:
        print('The start and end vertices are not the same.\n')
        sys.exit()
    if cost != 'infinite':
        if len(car_cycle) == 1:
            car_cycle = []
        else:
            car_cycle = get_edges_from_path(car_cycle[:-1]) + [(car_cycle[-2], car_cycle[-1])]
        if len(car_cycle) != 1:
            driving_cost = sum([G.edges[e]['weight'] for e in car_cycle]) * 2 / 3
        else:
            driving_cost = 0
        walking_cost = 0
        shortest = dict(nx.floyd_warshall(G))

        for drop_location in dropoffs:
            for house in dropoff_mapping[drop_location]:
                walking_cost += shortest[drop_location][house]
        cost = driving_cost + walking_cost
    return cost

In [ ]:
def nn2_solve(list_of_locations, list_of_homes, starting_car_location, adjacency_matrix, params=[]):
    start_idx = list_of_locations.index(starting_car_location)
    homes_idx = [list_of_locations.index(h) for h in list_of_homes]
    graph1, msg1 = adjacency_matrix_to_graph(adjacency_matrix)
    start_shortest_paths = nx.shortest_path(graph1, start_idx, target=None, weight='weight')
    minimum = float('inf')
    final_path = []
    final_dropoffs = {}
    for h in homes_idx:
        homes_copy = homes_idx.copy()
        path = []
        path.extend(start_shortest_paths[h])
        path.pop()
        drop_offs = {}
        if start_idx in homes_copy:
            drop_offs[start_idx] = [start_idx]
            homes_copy.remove(start_idx)
        current_idx = h
        while homes_copy:
            lst_dist = []
            shortest_path_length = nx.shortest_path_length(graph1, current_idx, target=None, weight='weight')
            for h in homes_copy:
                lst_dist.append((h, shortest_path_length[h]))
            idx, _ = min(lst_dist, key=lambda x: x[1])
            shortest_path = nx.shortest_path(graph1, current_idx, idx, 'weight')
            shortest_path.pop()
            path.extend(shortest_path)
            drop_offs[idx] = [idx]
            current_idx = idx
            homes_copy.remove(idx)
        last_shortest_path = nx.shortest_path(graph1, current_idx, start_idx, 'weight')
        path.extend(last_shortest_path)
        cost = cost_of_solution_no_print(graph1, path, drop_offs)
        if cost < minimum:
            final_path = path
            final_dropoffs = drop_offs
    return final_path, final_dropoffs

In [ ]:
data = read_file('test_inputs/27_50.in')
num_loc, num_house, lst_loc, lst_house, start_loc, adj_mat = data_parser(data)
nn2_solve(lst_loc, lst_house, start_loc, adj_mat)

## 2-Approximation using MST

- If the MST is connected then we can perform DFS and construct the path.
- If the MST is not connected then we need to connect all homes by finding shortest paths between them.
- To get the final path we look at each path in the list and check in the original graph, if there is an edge then we do nothing, but if it does not have an edge then we replace it with the shortest path between those nodes.

In [ ]:
def connect_homes(graph, nodes):
    newgraph = nx.Graph()
    newgraph.add_nodes_from(nodes)
    for i in range(len(nodes)):
        for j in range(i + 1, len(nodes)):
            w = nx.shortest_path_length(graph, nodes[i], nodes[j], 'weight')
            newgraph.add_edge(nodes[i], nodes[j], weight=w)
    return newgraph
    
def mst_solve(list_of_locations, list_of_homes, starting_car_location, adjacency_matrix, params=[]):
    start_idx = list_of_locations.index(starting_car_location)
    homes_idx = [list_of_locations.index(h) for h in list_of_homes]
    nodes = homes_idx + [start_idx]
    graph, msg = adjacency_matrix_to_graph(adjacency_matrix)
    newgraph = graph.subgraph(nodes)
    if not nx.is_connected(newgraph):
        newgraph = connect_homes(graph, nodes)
    mst = nx.minimum_spanning_tree(newgraph, 'weight')
    dfs = list(nx.dfs_edges(mst, start_idx))
    path = [start_idx]
    drop_offs = {}
    if start_idx in homes_idx:
        drop_offs[start_idx] = [start_idx]
    current_idx = -1
    while dfs:
        edge = dfs.pop(0)
        current_idx = edge[1]
        if current_idx not in path:
            path.append(current_idx)
            drop_offs[current_idx] = [current_idx]
    final_path = [path[0]]
    for i in range(len(path)-1):
        if graph.has_edge(path[i], path[i+1]):
            final_path.append(path[i+1])
        else:
            final_path.pop()
            final_path.extend(nx.shortest_path(graph, path[i], path[i+1], 'weight'))
    last_shortest_path = nx.shortest_path(graph, final_path[-1], start_idx, 'weight')
    final_path.pop()
    final_path.extend(last_shortest_path)
    return final_path, drop_offs

In [ ]:
# data = read_file('test_inputs/1_50.in')
data = read_file('inputs/79_200.in')
num_loc, num_house, lst_loc, lst_house, start_loc, adj_mat = data_parser(data)
mst_solve(lst_loc, lst_house, start_loc, adj_mat)

## Branch-and-bound using the algorithm in the textbook (NOT WORKING)

In [ ]:
NUM_DEC = 5  # number of decimal numbers for rounding

In [ ]:
#code

# list_of_locations, list_of_homes, starting_car_location, adjacency_matrix, 
def add_weights2(g, lower, upper):
    """Update the weight of each edge to be a random weight within the bounds."""
    for edge, data in g.edges.items():
        w = rd.uniform(lower, upper)
        data['weight'] = round(w, NUM_DEC)
        
def print_edge_weights(g, n=5):
    for edge, data in g.edges.items():
        if n <= 0:
            break
        n-=1
        print(edge, data)
        
def is_complete_graph(g):
    n = len(g.nodes)
    return (n * (n-1) / 2) == len(g.edges())

def convert_to_complete_graph(g, starting_car_location, list_of_homes, list_of_locations):
    """return a complete graph with homes and start loc."""
    rs = nx.floyd_warshall(g, weight='weight')
    result_g = nx.Graph()
    start_idx = list_of_locations.index(starting_car_location)
    homes_idx = [list_of_locations.index(h) for h in list_of_homes]
    subgraph_idx = list(set([start_idx] + homes_idx)) # incase starting position is home
    
    subgraph = g.subgraph(subgraph_idx)
    subgraph = subgraph.copy()
    for i in range(len(subgraph_idx)):
        for j in range(i+1, len(subgraph_idx)):
            v0 = subgraph_idx[i]
            v1 = subgraph_idx[j]
            if not subgraph.has_edge(v0, v1):
                subgraph.add_edge(v0, v1, weight = rs[v0][v1])
    return subgraph

def lowerbound1(g, V, S, a, b):
#     print("lowerbound1: S(current path) = ", S)
    V_set = set(V)
    S_set = set(S)
    V_minus_S = V_set.difference(S_set)
    
    cur_min_weight_a = float('inf')
    cur_min_weight_b = float('inf')
    
     # the lighest edge from node to V - S
    def find_min_weight(node):
        # COMMENT BELOW BECAUSE WE ARE DEALING WITH COMPLETE GRAPH node always has neighbors
#         node_nb_set = set(g.neighbors(node))
#         node_nb_intersect_V_minus_S = node_nb_set.intersection(V_minus_S)
#         cur_min_weight_node =  float('inf')
# #         print("for node", node, "its neighbors", node_nb_intersect_V_minus_S)
#         for node_nb_V_minus_S in node_nb_intersect_V_minus_S:
#             cur_min_weight_node = min(cur_min_weight_node,  g.edges[(node, node_nb_V_minus_S)]['weight'])
#         return cur_min_weight_node
    # *************************************************************
        cur_min_weight_node =  float('inf')
#         print("for node", node, "its neighbors", node_nb_intersect_V_minus_S)
        for nb in V_minus_S:
            cur_min_weight_node = min(cur_min_weight_node,  g.edges[(node, nb)]['weight'])
        return cur_min_weight_node

 
    # the lighest edge from a to V - S
    cur_min_weight_a = find_min_weight(a)
    
    # the lighest edge from b to V - S
    cur_min_weight_b = find_min_weight(b)
    
    # find the sum of weights of MST 
    V_minus_S_subgraph = g.subgraph(list(V_minus_S))
    
    mst = nx.minimum_spanning_tree(V_minus_S_subgraph)
    sum_weight_mst = sum([data['weight'] for edge, data in mst.edges.items()])
    
    assert cur_min_weight_a != float('inf') and cur_min_weight_b != float('inf')
    return cur_min_weight_a + cur_min_weight_b + sum_weight_mst

def branch_and_bound(g, starting_vertex, lower_bound):
    active_subproblems = PriorityQueue()
#     active_subproblems = []
#     active_subproblems.append((starting_vertex, [starting_vertex], starting_vertex))
    active_subproblems.put((lower_bound(g, g.nodes(), [starting_vertex], starting_vertex, starting_vertex), (starting_vertex, [starting_vertex], starting_vertex)))
    best_so_far = float('inf')
    best_path = []
    while active_subproblems.qsize() > 0:
#     while len(active_subproblems) > 0:
        # choose
        active_sub = active_subproblems.get()[1]
#         active_sub = active_subproblems.pop()
#         print("active_sub=", active_sub)
        # expand
        V_set = set(g.nodes)
        S_set = set(tuple(active_sub[1]))
        V_minus_S = V_set.difference(S_set)
        
        for x in V_minus_S:
            path = active_sub[1]+[x]
            P = (active_sub[0], path , x)
            # complete sol
            total_w = 0
            
            
            if len(path) == len(g.nodes):
                for i in range(len(path)-1):
                    total_w += g.edges[(path[i], path[i+1])]['weight']
                if total_w < best_so_far:
                    best_so_far = total_w
                    best_path = path
        
           
            # if lowerbound < bestsofar add Pi to S 
            else:
                cur_lower_bound = lower_bound(g, g.nodes, path, active_sub[0], x)
                if cur_lower_bound < best_so_far:
                    active_subproblems.put((cur_lower_bound, P))
#                     active_subproblems.append(P)
                    
                #(g, V, S, a, b): 
    return best_so_far, best_path

In [ ]:
#test
G1 = nx.complete_graph(10)
add_weights2(G1, 1, 100000000)
nx.draw(G1, node_color='yellow', with_labels=True)

# print_edge_weights(G1, len(G1.edges()))

# Branch-and-bound using permutation (brute force)

In [ ]:
def convert_to_complete_graph(g, starting_car_location, list_of_homes, list_of_locations):
    """return a complete graph with homes and start loc."""
    rs = nx.floyd_warshall(g, weight='weight')
    result_g = nx.Graph()
    start_idx = list_of_locations.index(starting_car_location)
    homes_idx = [list_of_locations.index(h) for h in list_of_homes]
    subgraph_idx = list(set([start_idx] + homes_idx)) # incase starting position is home
    
    subgraph = g.subgraph(subgraph_idx)
    subgraph = subgraph.copy()
    for i in range(len(subgraph_idx)):
        for j in range(i+1, len(subgraph_idx)):
            v0 = subgraph_idx[i]
            v1 = subgraph_idx[j]
            if not subgraph.has_edge(v0, v1):
                subgraph.add_edge(v0, v1, weight = rs[v0][v1])
    return subgraph

def get_cost_of_path(g, path):
    total = 0
    for i in range(len(path)-1):
        total += g.edges[(path[i], path[i+1])]['weight']
    return total

def all_walk_homes(start_idx, homes_idx):
    car_path = [start_idx]
    drop_off_locations = {start_idx: homes_idx}
    return car_path, drop_off_locations

def perm_shortest_cycle(g, start_idx, homes_idx, max_len = 10):
    #g is [start, h1, h2, h3]
    #append start_idx to the first and last place if home is not start loc
    # if start is home, only append last
    
    g_no_start = list(g.nodes)
    g_no_start.remove(start_idx)
    perm = permutations(g_no_start) # without the start_idx
    
    # stop if we think it takes too long to run this(permutation)
    if len(g_no_start) > max_len:
        return None
    print("bb_solve")
    cur_min = float("inf")
    cur_min_path = []
    
    for p in perm:
        p = list(p)
        p.insert(0, start_idx)
        p.append(start_idx)
        cur_cost = get_cost_of_path(g, p)

        if cur_cost < cur_min:
            cur_min = cur_cost
            cur_min_path = p
    return cur_min_path

def reconstruct_cycle_from_complete_homes_start_graph(original_g, cycle, start_idx, homes_idx):
    car_path = []
    drop_off_locations = {}
    
    cur_idx = cycle.pop(0)
    if cur_idx in homes_idx:
        drop_off_locations[cur_idx] = [cur_idx]
    while cycle:
        next_idx = cycle.pop(0)
        s_path = nx.shortest_path(original_g, cur_idx, next_idx, weight='weight')
        
        if len(cycle) != 0: #don't drop off when back to start
            drop_off_locations[next_idx] = [next_idx]
        s_path.pop()
        car_path.extend(s_path)
        cur_idx = next_idx

    car_path.extend(nx.shortest_path(original_g, cur_idx, start_idx, weight='weight'))
    print(car_path)
    print(drop_off_locations)
    return car_path, drop_off_locations
    
def branch_and_bound_perm(list_of_locations, list_of_homes, starting_car_location, adjacency_matrix, params=[]):
    PERM_LEN = 10
    
    start_idx = list_of_locations.index(starting_car_location)
    homes_idx = [list_of_locations.index(h) for h in list_of_homes]
    g, msg1 = adjacency_matrix_to_graph(adjacency_matrix)

    check_len = 0
    if start_idx in homes_idx:
        check_len = len(homes_idx) - 1
    else:
        check_len = len(homes_idx)
        
    if check_len > PERM_LEN:
        return all_walk_homes(start_idx, homes_idx)
        
    car_path = []
    drop_off_locations = {}
    
    complete_g = convert_to_complete_graph(g, starting_car_location, list_of_homes, list_of_locations)
    
    shortest_homes_cycle = perm_shortest_cycle(complete_g, start_idx, homes_idx, PERM_LEN)# check len again
    if not shortest_homes_cycle:
        return all_walk_homes(start_idx, homes_idx)
    else:
        return reconstruct_cycle_from_complete_homes_start_graph(g, shortest_homes_cycle, start_idx, homes_idx)

In [ ]:
data = read_file('test_inputs/216_50.in') # 116 216 294 52
num_loc, num_house, lst_loc, lst_house, start_loc, adj_mat = data_parser(data)
car_path, drop_off_locations = branch_and_bound_perm(lst_loc, lst_house, start_loc, adj_mat)

start_idx = lst_loc.index(start_loc)
homes_idx = [lst_loc.index(h) for h in lst_house]
g, msg1 = adjacency_matrix_to_graph(adj_mat)
nx.draw(g, node_color='yellow', with_labels=True)
g, msg1 = adjacency_matrix_to_graph(adj_mat)
cost_of_solution(g, car_path, drop_off_locations)

# Rao drives TAs to some dropoff locations (clustering)

In [ ]:
data = read_file('inputs/79_200.in')
num_loc, num_house, list_of_locations, list_of_homes, starting_car_location, adjacency_matrix = data_parser(data)
graph, msg = adjacency_matrix_to_graph(adjacency_matrix)
start_idx = list_of_locations.index(starting_car_location)
homes_idx = [list_of_locations.index(h) for h in list_of_homes]

plt.figure(1)
nx.draw(graph, with_labels=True)

#first compute the best partition
partition = community.best_partition(graph)

#drawing
plt.figure(2)
size = float(len(set(partition.values())))
pos = nx.spring_layout(graph)
count = 0.
for com in set(partition.values()) :
#     print(type(com))
    count = count + 1.
    list_nodes = [nodes for nodes in partition.keys()
                                if partition[nodes] == com]
    nx.draw_networkx_nodes(graph, pos, list_nodes, node_size = 20,
                                node_color = str(count / size))
print('Number of communities: ', count)

nx.draw_networkx_edges(graph, pos, alpha=0.5)
plt.show()

In [ ]:
def count_homes(k, graph, homes_idx):
    lst = []
    for node in graph.nodes():
        lst_homes = []
        neighbors = list(graph.neighbors(node))
        for n in neighbors:
            if n in homes_idx:
                lst_homes.append(n)
        if k == 2:
            for n in neighbors:
                for nn in graph.neighbors(n):
                    if nn in homes_idx:
                        lst_homes.append(nn)
        if len(lst_homes) > 0:
            lst.append((node, lst_homes))
    return lst

def init_pq(lst, graph, cur_idx):
    shortest_paths = nx.shortest_path_length(graph, cur_idx, weight='weight')
    pq = PriorityQueue()
    for i in lst:
        node = i[0]
        pq.put((shortest_paths[node], i))
    return pq
    
def k_layers_cluster(k, list_of_locations, list_of_homes, starting_car_location, adjacency_matrix):
    start_idx = list_of_locations.index(starting_car_location)
    homes_idx = [list_of_locations.index(h) for h in list_of_homes]
    graph, msg = adjacency_matrix_to_graph(adjacency_matrix)
    path = []
    drop_offs = {}
    lst = count_homes(k, graph, homes_idx)
    cur_idx = start_idx
    while homes_idx:
        pq = init_pq(lst, graph, cur_idx)
        cost, center_homes = pq.get()
        center = center_homes[0]
        lst_homes = center_homes[1]
        lst_copy = lst_homes.copy()
        for h in lst_copy:
            if h in homes_idx:
                homes_idx.remove(h)
            else:
                lst_homes.remove(h)
        if lst_homes:
            shortest_path = nx.shortest_path(graph, cur_idx, center, 'weight')
            shortest_path.pop()
            path.extend(shortest_path)
            drop_offs[center] = lst_homes
            cur_idx = center
        lst.remove(center_homes)
    last_shortest_path = nx.shortest_path(graph, cur_idx, start_idx, 'weight')
    path.extend(last_shortest_path)
    return path, drop_offs
    
def nn_cluster_solve(list_of_locations, list_of_homes, starting_car_location, adjacency_matrix, params=[]):
    return k_layers_cluster(2, list_of_locations, list_of_homes, starting_car_location, adjacency_matrix)

In [ ]:
data = read_file('inputs/79_200.in')
num_loc, num_house, list_of_locations, list_of_homes, starting_car_location, adjacency_matrix = data_parser(data)
nn_cluster_solve(list_of_locations, list_of_homes, starting_car_location, adjacency_matrix)

# Clustering 2

In [ ]:
def count_homes(k, graph, homes_idx):
    lst = []
    for node in graph.nodes():
        lst_homes = []
        neighbors = list(graph.neighbors(node))
        for n in neighbors:
            if n in homes_idx:
                lst_homes.append(n)
        if k == 2:
            for n in neighbors:
                for nn in graph.neighbors(n):
                    if nn in homes_idx:
                        lst_homes.append(nn)
        if len(lst_homes) > 0:
            lst.append((node, lst_homes))
    return lst

def init_pq(lst, graph, cur_idx):
    shortest_paths = nx.shortest_path_length(graph, cur_idx, weight='weight')
    pq = PriorityQueue()
    for i in lst:
        node = i[0]
        pq.put((shortest_paths[node], i))
    return pq
    
def k_layers_cluster2(k, list_of_locations, list_of_homes, starting_car_location, adjacency_matrix):
    start_idx = list_of_locations.index(starting_car_location)
    homes_idx = [list_of_locations.index(h) for h in list_of_homes]
    graph, msg = adjacency_matrix_to_graph(adjacency_matrix)
    path = []
    drop_offs = {}
    lst = count_homes(k, graph, homes_idx)
    cur_idx = start_idx
    while homes_idx:
        pq = init_pq(lst, graph, cur_idx)
        cost, center_homes = pq.get()
        center = center_homes[0]
        lst_homes = center_homes[1]
        lst_copy = lst_homes.copy()
        for h in lst_copy:
            if h in homes_idx:
                homes_idx.remove(h)
            else:
                lst_homes.remove(h)
        if lst_homes:
            if len(lst_homes) == 1:
                center = lst_homes[0]
            shortest_path = nx.shortest_path(graph, cur_idx, center, 'weight')
            shortest_path.pop()
            path.extend(shortest_path)
            if center in drop_offs:
                drop_offs[center] += lst_homes
            else:
                drop_offs[center] = lst_homes
            cur_idx = center
        lst.remove(center_homes)
    last_shortest_path = nx.shortest_path(graph, cur_idx, start_idx, 'weight')
    path.extend(last_shortest_path)
    return path, drop_offs
    
def nn_cluster_solve2(list_of_locations, list_of_homes, starting_car_location, adjacency_matrix, params=[]):
    return k_layers_cluster2(1, list_of_locations, list_of_homes, starting_car_location, adjacency_matrix)

In [ ]:
data = read_file('inputs/79_200.in')
num_loc, num_house, list_of_locations, list_of_homes, starting_car_location, adjacency_matrix = data_parser(data)
G, msg = adjacency_matrix_to_graph(adjacency_matrix)
car_path, dropoffs = nn_cluster_solve2(list_of_locations, list_of_homes, starting_car_location, adjacency_matrix)
print(car_path)
print(dropoffs)
print(cost_of_solution_no_print(G, car_path, dropoffs))

# Greedy Clustering

In [ ]:
def greedy_clustering_solver(list_of_locations, list_of_homes, starting_car_location, adjacency_matrix, params=[]):
    # Count adjacent homes for each vertice and add them to the PQ
    start_idx = list_of_locations.index(starting_car_location)
    homes_idx = [list_of_locations.index(h) for h in list_of_homes]
    g, msg1 = adjacency_matrix_to_graph(adjacency_matrix)
    PQ = PriorityQueue()
    car_path = []
    drop_off_locations = {}
    
    # initilize PQ
    for v in g.nodes():
        homes = []
        for nb in g.neighbors(v):
            if nb in homes_idx:
                homes.append(nb)
        PQ.put((len(homes), (v, homes)))
    cur_idx = start_idx
    while homes_idx:
        (dropoff_loc, dropoff_homes) = PQ.get()[1]
        dropoff_copy = dropoff_homes.copy()
        for h in dropoff_copy:
            if h in homes_idx:
                homes_idx.remove(h)
            else:
                dropoff_homes.remove(h)
        if dropoff_homes:
            s_path = nx.shortest_path(g, cur_idx, dropoff_loc, weight='weight')
            cur_idx = dropoff_loc
            s_path.pop()
            car_path.extend(s_path)
            drop_off_locations[dropoff_loc] = dropoff_homes
    
    car_path.extend(nx.shortest_path(g, cur_idx, start_idx, weight='weight'))
    return car_path, drop_off_locations   

In [ ]:
data = read_file('test_inputs/27_200.in')
num_loc, num_house, lst_loc, lst_house, start_loc, adj_mat = data_parser(data)
car_path, drop_off_locations = greedy_clustering_solver(lst_loc, lst_house, start_loc, adj_mat)

g, msg1 = adjacency_matrix_to_graph(adj_mat)
print(car_path)
print(cost_of_solution(g, car_path, drop_off_locations))

In [ ]:
data = read_file('test_inputs/116_50.in')
num_loc, num_house, lst_loc, lst_house, start_loc, adj_mat = data_parser(data)
car_path, drop_off_locations = greedy_clustering_solver(lst_loc, lst_house, start_loc, adj_mat)

g, msg1 = adjacency_matrix_to_graph(adj_mat)
print(car_path)
print(cost_of_solution(g, car_path, drop_off_locations))

# Local Search

In [ ]:
def get_shortest_cost(car_path, shortest_paths):
    total = 0
    for i in range(len(car_path)-1):
        total += (shortest_paths[car_path[i]][car_path[i+1]] * 2 / 3) 
    return total
    
def inverse(car_path, i, j):
    """swap anything in between i and j, both indices inclusively"""
    return car_path[:i] + car_path[i:j+1][::-1] + car_path[j+1:]

def insert(car_path, i, j):
    p = car_path[:]
    item = p.pop(j)
    p.insert(i, item)
    return p

def swap(car_path, i, j):
    p = car_path[:]
    p[i], p[j] = p[j], p[i]
    return p

def get_valid_dropoffs(car_path, homes_idx):
    drop_off_locations = {}
    for v in car_path:
        if v in homes_idx:
            drop_off_locations[v] = [v]
    return drop_off_locations

def get_local_best_neighbor(car_path, shortest_paths, i ,j):
    """Minimum of three modified paths, original one not included"""
    p1 = inverse(car_path, i, j)
    p2 = insert(car_path, i, j)
    p3 = swap(car_path, i, j)
    cost_dict = {}
    cost_dict["p1"] = get_shortest_cost(p1, shortest_paths)
    cost_dict["p2"] = get_shortest_cost(p2, shortest_paths)
    cost_dict["p3"] = get_shortest_cost(p3, shortest_paths)
    minKey = min(cost_dict, key=cost_dict.get)
    if minKey == "p1":
        return p1, cost_dict["p1"]
    elif minKey == "p2":
        return p2, cost_dict["p2"]
    else:
        return p3, cost_dict["p3"]

def reconstruct_full_cycle_from_incomplete_cycle(original_g, cycle, start_idx, homes_idx):
    """fill in the vertices betwee each pair of vertices in cycle based on the original graph"""
    car_path = []
    drop_off_locations = {}
    
    cur_idx = cycle.pop(0)
    if cur_idx in homes_idx:
        drop_off_locations[cur_idx] = [cur_idx]
    while cycle:
        next_idx = cycle.pop(0)
        s_path = nx.shortest_path(original_g, cur_idx, next_idx, weight='weight')
        
        if len(cycle) != 0: #don't drop off when back to start
            drop_off_locations[next_idx] = [next_idx]
        s_path.pop()
        car_path.extend(s_path)
        cur_idx = next_idx

    car_path.extend(nx.shortest_path(original_g, cur_idx, start_idx, weight='weight'))
    return car_path, drop_off_locations 

In [ ]:
# L_MAX = 120
L_MAX = 60
INIT_TEMP = 18
ITER_NUM = 1000
INIT_PROB = 0.5

def init_temp_list(car_path, shortest_paths, length):
    L = PriorityQueue()
    i = 0
    while i < length:
        neighbor, costy = rand_gen_neighbor(car_path, shortest_paths)
        costx = get_shortest_cost(car_path, shortest_paths)
        if costy < costx:
            car_path = neighbor
        else:
            t = -abs(costy - costx) / math.log(INIT_PROB)
            L.put(-t)
            i = i + 1
    return L

def gen_rand_idx(x, y):
    while True:
        a = rd.randint(x, y)
        b = rd.randint(x, y)
        if abs(a-b) >= 2:
            if a < b:
                return a, b
            else:
                return b, a
            
def rand_gen_neighbor(car_path, shortest_paths):
    i, j = gen_rand_idx(1, len(car_path)-2)
    return get_local_best_neighbor(car_path, shortest_paths, i, j)

def acceptance_probability(delta, t_max):
    try:
        return math.exp(-delta / t_max)
    except ZeroDivisionError:
        return 0
    except OverflowError:
        return 0

def new_temp(t, delta, r):
    return (t - delta) / math.log(r)

def local_search(car_path, shortest_paths):
    L = init_temp_list(car_path, shortest_paths, L_MAX)
    k = 0
    while k < L_MAX - 1:
        t_max = -L.get(block=False)
        k = k + 1
        t = 0
        c = 0
        m = 0
        while m < ITER_NUM:
            m = m + 1
            neighbor, costy = rand_gen_neighbor(car_path, shortest_paths)
            costx = get_shortest_cost(car_path, shortest_paths)
            if costy < costx:
                car_path = neighbor
            else:
                delta = costy - costx
                p = acceptance_probability(delta, t_max)
                r = rd.random()
                if r < p:
                    t = new_temp(t, delta, r)
                    c = c + 1
        if c != 0:
            L.get(block=False)
            L.put(-t/c)
    return car_path
    
def local_search_solver(list_of_locations, list_of_homes, starting_car_location, adjacency_matrix, input_file, params=[]):
    homes_idx = [list_of_locations.index(h) for h in list_of_homes]
    graph, msg = student_utils.adjacency_matrix_to_graph(adjacency_matrix)
    
    nn_output_directory = 'nearest_neighbor_algo/outputs'
    nn_output_file = utils.input_to_output(input_file, nn_output_directory)
    nn_output_data = utils.read_file(nn_output_file)
    car_cycle = nn_output_data[0]
    car_cycle_idx = student_utils.convert_locations_to_indices(car_cycle, list_of_locations)

    shortest_paths = nx.floyd_warshall(graph, weight='weight')
    new_car_cycle_idx = local_search(car_cycle_idx, shortest_paths)

    #final_path
    final_cycle = []
    for i in range(len(new_car_cycle_idx)-1):
        shortest_path = nx.shortest_path(graph, new_car_cycle_idx[i], new_car_cycle_idx[i+1], weight='weight')
        shortest_path.pop()
        final_cycle.extend(shortest_path)
    final_cycle.append(new_car_cycle_idx[-1])
    dropoffs = get_valid_dropoffs(final_cycle, homes_idx)
    return final_cycle, dropoffs